# 📚 Import Libraries

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

# 📄 Load Datasets

In [2]:
# Load the main dataset
df = pd.read_csv(
    'Data.csv',
    sep=';',
    quotechar='"',
    on_bad_lines='skip',
    encoding='utf-8',
    dtype={
        'UNIQUE_KEY': str,
        'REVENUES': str,
        'TOTAL ASSETS': str
    },
    low_memory=False
)

df.head()

,UNIQUE_KEY,NAME,YEAR,CITY,COUNTRY,STATUS,LEGAL_FORM,ISIC,REGISTRATION,LIQUIDATION,TERMINATION,REVENUES,TOTAL ASSETS
0,"1,29076E+11",Karl Roth mechanische Werkstätte GmbH,2024.0,fürth,DE,active,GmbH,2592.0,1989.0,NaN,NaN,990.000.000.000.000,865.389.500.000.000
1,"1,20775E+13",VACOM Vakuum Komponenten & Messtechnik GmbH,2024.0,großlöbichau,DE,active,GmbH,2651.0,1993.0,NaN,NaN,58.401.908.050.000.000,56.201.332.840.000.000
2,1339576043,Landwirtschaftliche Erzeugergemeinschaft Berka...,2024.0,bismark (altmark),DE,active,eG,150.0,1991.0,NaN,NaN,2.400.000.000.000.000,81.183.970.000.000
3,"1,15311E+11",Immobilienverwaltung Ottersbach GmbH & Co. KG,2024.0,bonn,DE,active,GmbH & Co. KG,6820.0,2015.0,NaN,NaN,710.000.000.000.000,3.213.887.520.000.000
4,"1,13539E+11",Bärhausen GmbH & Co. KG,2024.0,lauterbach,DE,active,GmbH & Co. KG,2822.0,1965.0,NaN,NaN,4.500.000.000.000.000,2.404.823.060.000.000


In [3]:
# Load the reference Business Type table
business_type = pd.read_excel('BUSINESS_TYPE.xlsx')
business_type

,Legal_Form,Business_Type
0,AG – Aktiengesellschaft,Public
1,SE – Societas Europaea,Public
2,KGaA – Kommanditgesellschaft auf Aktien,Hybrid
3,GmbH – Gesellschaft mit beschränkter Haftung,Private
4,UG (haftungsbeschränkt),Private
5,GmbH & Co. KG,Hybrid
6,AG & Co. KG,Hybrid
7,SE & Co. KG,Hybrid
8,OHG – Offene Handelsgesellschaft,Private
9,KG – Kommanditgesellschaft,Private


# 🔎 Quality Check

## Check LEGAL_FORM consistency

In [4]:
# Check unqiue values
# pd.set_option('display.max_rows', None)
df['LEGAL_FORM'].value_counts(dropna=False).sort_index()

LEGAL_FORM
A/S                25
A/S & Co. KG        5
AB                 26
AB & Co. KG         5
AG               5495
                ...  
ΟΕ                  3
ООО                 2
ПАО                 1
בע"מ                1
NaN             85493
Name: count, Length: 199, dtype: int64

In [6]:
# Check for missing values
missing_count = df['LEGAL_FORM'].isnull().sum()
total_count = len(df)
missing_percentage = (missing_count / total_count) * 100

print(f"Missing values in LEGAL_FORM: {missing_count:,} ({missing_percentage:.2f}%)")

Missing values in LEGAL_FORM: 85,493 (8.55%)


# 🧹 Data Cleansing

## Clean and prepare the Business Type reference table

In [7]:
# Step 1: Make a copy table
ref = business_type.copy()

# Step 2: Clean and simplify the legal forms
ref['short_form'] = ref['Legal_Form'].str.split('–').str[0].str.strip()
ref['short_form'] = ref['short_form'].replace({'UG (haftungsbeschränkt)': 'UG'})

# Step 3: Remove the combined 'e.K. / e.Kfm. / e.Kfr.' entry
ref = ref[ref['short_form'] != 'e.K. / e.Kfm. / e.Kfr.']

# Step 4: Add manual entries for expanded forms
expanded_forms = {
    'e. K.': 'Private',
    'e. Kfm.': 'Private',
    'e. Kfr.': 'Private'
}

manual_df = pd.DataFrame({
    'short_form': list(expanded_forms.keys()),
    'Business_Type': list(expanded_forms.values()),
    'Legal_Form': [None] * len(expanded_forms)  # Or use 'Manual entry'
})

# Step 5: Combine cleaned reference and manual rows
ref_cleaned = pd.concat([ref[['Legal_Form', 'short_form', 'Business_Type']], manual_df], ignore_index=True)
ref_cleaned

,Legal_Form,short_form,Business_Type
0,AG – Aktiengesellschaft,AG,Public
1,SE – Societas Europaea,SE,Public
2,KGaA – Kommanditgesellschaft auf Aktien,KGaA,Hybrid
3,GmbH – Gesellschaft mit beschränkter Haftung,GmbH,Private
4,UG (haftungsbeschränkt),UG,Private
5,GmbH & Co. KG,GmbH & Co. KG,Hybrid
6,AG & Co. KG,AG & Co. KG,Hybrid
7,SE & Co. KG,SE & Co. KG,Hybrid
8,OHG – Offene Handelsgesellschaft,OHG,Private
9,KG – Kommanditgesellschaft,KG,Private


## Create final lookup dictionary

In [8]:
lookup_dict = dict(zip(ref_cleaned['short_form'], ref_cleaned['Business_Type']))
lookup_dict

{'AG': 'Public',
 'SE': 'Public',
 'KGaA': 'Hybrid',
 'GmbH': 'Private',
 'UG': 'Private',
 'GmbH & Co. KG': 'Hybrid',
 'AG & Co. KG': 'Hybrid',
 'SE & Co. KG': 'Hybrid',
 'OHG': 'Private',
 'KG': 'Private',
 'GbR': 'Private',
 'PartG': 'Private',
 'PartGmbB': 'Private',
 'eG': 'Hybrid',
 'Stiftung': 'Private',
 'VVaG': 'Hybrid',
 'EWIV': 'Hybrid',
 'Freiberufler': 'Private',
 'Other': 'Private',
 'e. K.': 'Private',
 'e. Kfm.': 'Private',
 'e. Kfr.': 'Private'}

# ✅ Adding Match Quality

## 1. Determine MATCH_QUALITY

In [40]:
def assign_match_quality(df, business_type_lookup):
    def get_match_quality(legal_form):
        if pd.isna(legal_form) or str(legal_form).strip() == '':
            return '❓ Missing'

        legal_form_str = str(legal_form).strip()

        # Exact match (case sensitive)
        if legal_form_str in business_type_lookup:
            return '✅ Exact'

        # Case-insensitive match
        for key in business_type_lookup:
            if key.lower() == legal_form_str.lower():
                return '✅ Exact'

        return '❌ Unmatched'

    df_enriched = df.copy()
    df_enriched['MATCH_QUALITY'] = df_enriched['LEGAL_FORM'].apply(get_match_quality)
    return df_enriched

In [41]:
# Apply to dataset using the cleaned dictionary
df_enriched = assign_match_quality(df, lookup_dict)

# Show preview of results
preview_cols = ['NAME', 'LEGAL_FORM', 'MATCH_QUALITY']
available_cols = [col for col in preview_cols if col in df_enriched.columns]

df_enriched[available_cols].head(10)

,NAME,LEGAL_FORM,MATCH_QUALITY
0,Karl Roth mechanische Werkstätte GmbH,GmbH,✅ Exact
1,VACOM Vakuum Komponenten & Messtechnik GmbH,GmbH,✅ Exact
2,Landwirtschaftliche Erzeugergemeinschaft Berka...,eG,✅ Exact
3,Immobilienverwaltung Ottersbach GmbH & Co. KG,GmbH & Co. KG,✅ Exact
4,Bärhausen GmbH & Co. KG,GmbH & Co. KG,✅ Exact
5,R & S Szilagyi Grundstücksverwertungsgesellsch...,GmbH,✅ Exact
6,fischer/collegen Bietigheim-Bissingen GmbH & C...,GmbH & Co. KG,✅ Exact
7,Wirtschaftsprüfungsgesellschaft Frankfurt GmbH,GmbH,✅ Exact
8,DATA MODUL AG Produktion und Vertrieb von elek...,AG,✅ Exact
9,Hermann-Lietz-Schule Spiekeroog gemeinnützige ...,gGmbH,❌ Unmatched


### Inspect the results

In [11]:
# Filter by match quality
exact_matches = df_enriched[df_enriched['MATCH_QUALITY'] == '✅ Exact']
print(f"✅ Exact matches: {len(exact_matches):,} records")

missing_matches = df_enriched[df_enriched['MATCH_QUALITY'] == '❓ Missing']
print(f"❓ Missing legal forms: {len(missing_matches):,} records")

unmatched = df_enriched[df_enriched['MATCH_QUALITY'] == '❌ Unmatched']
print(f"❌ Unmatched records: {len(unmatched):,} records")

✅ Exact matches: 694,673 records
❓ Missing legal forms: 85,493 records
❌ Unmatched records: 219,801 records


In [12]:
# Show summary statistics
print(f"Total records processed: {len(df_enriched):,}")

print("\nMatch Quality Distribution:")
match_stats = df_enriched['MATCH_QUALITY'].value_counts()
for match_type, count in match_stats.items():
    percentage = (count / len(df_enriched)) * 100
    print(f"  {match_type}: {count:,} ({percentage:.1f}%)")

Total records processed: 999,967

Match Quality Distribution:
  ✅ Exact: 694,673 (69.5%)
  ❌ Unmatched: 219,801 (22.0%)
  ❓ Missing: 85,493 (8.5%)


## 2. Analyze unmatched patterns

In [14]:
# Display unmatched record
unmatched[['NAME', 'LEGAL_FORM', 'MATCH_QUALITY']]

,NAME,LEGAL_FORM,MATCH_QUALITY
9,Hermann-Lietz-Schule Spiekeroog gemeinnützige ...,gGmbH,❌ Unmatched
76,CIT Group Germany AG & Co. OHG,AG & Co. OHG,❌ Unmatched
150,Diakonissenkrankenhaus Dessau gemeinnützige GmbH,gGmbH,❌ Unmatched
185,Gillette Deutschland GmbH & Co. OHG,GmbH & Co. OHG,❌ Unmatched
245,Amani gUG,gUG,❌ Unmatched
...,...,...,...
999541,Alumni des Freiherr-vom-Stein-Gymnasiums e. V.,e. V.,❌ Unmatched
999542,Mein Sport Schleswig-Holstein e. V.,e. V.,❌ Unmatched
999548,"Förderverein St. Josef, Mönchengladbach-Hermge...",e. V.,❌ Unmatched
999550,Waldkindergarten Waldfrüchtchen e. V.,e. V.,❌ Unmatched


In [15]:
 # Most common unmatched legal forms
print(f"\n TOP 15 MOST COMMON UNMATCHED LEGAL FORMS:")

unmatched_forms = unmatched['LEGAL_FORM'].value_counts().head(15)
for form, count in unmatched_forms.items():
    percentage = (count / len(unmatched)) * 100
    print(f"  '{form}': {count:,} records ({percentage:.1f}%)")


 TOP 15 MOST COMMON UNMATCHED LEGAL FORMS:
  'e. V.': 183,750 records (83.6%)
  'eGbR': 17,582 records (8.0%)
  'UG & Co. KG': 4,641 records (2.1%)
  'Ltd.': 2,627 records (1.2%)
  'PartG mbB': 2,330 records (1.1%)
  'gGmbH': 1,996 records (0.9%)
  'Ltd. & Co. KG': 1,710 records (0.8%)
  'EI': 1,426 records (0.6%)
  'gUG': 860 records (0.4%)
  'GmbH & Co. OHG': 446 records (0.2%)
  'B.V.': 382 records (0.2%)
  'AG & Co. OHG': 240 records (0.1%)
  'Association': 131 records (0.1%)
  'gesellschaft mbH': 122 records (0.1%)
  'KG & Co. KG': 119 records (0.1%)


In [16]:
# Word frequency analysis
print(f"\n🔍 WORD FREQUENCY ANALYSIS:")
all_text = ' '.join(unmatched['LEGAL_FORM'].astype(str).tolist()).lower()
words = re.findall(r'\b\w+\b', all_text)
word_counts = Counter(words)

print("Most common words in unmatched forms:")
for word, count in word_counts.most_common(12):
    if len(word) > 1:  # Skip single letters
        percentage = (count / len(words)) * 100
        print(f"  '{word}': {count:,} times ({percentage:.1f}%)")


🔍 WORD FREQUENCY ANALYSIS:
Most common words in unmatched forms:
  'egbr': 17,584 times (4.2%)
  'co': 7,771 times (1.8%)
  'kg': 7,055 times (1.7%)
  'ug': 4,656 times (1.1%)
  'ltd': 4,338 times (1.0%)
  'partg': 2,330 times (0.6%)
  'mbb': 2,330 times (0.6%)
  'ggmbh': 1,998 times (0.5%)
  'ei': 1,426 times (0.3%)
  'gug': 860 times (0.2%)


## 3. Handling Unmatched Legal Forms: Partial vs Truly Unmatched

To improve classification, we reviewed the top unmatched `LEGAL_FORM` entries and determined which ones can reasonably be mapped to known forms (**🔍 Partial**) and which should remain **❌Unmatched**.

---

#### ✅ Probably Partial Matches:

| Unmatched Form       | Suggested Match | Reason |
|----------------------|------------------|--------|
| `eGbR`               | GbR              | Typo with "e" prefix; likely GbR |
| `PartG mbB`          | PartGmbB         | Variant spelling |
| `gGmbH`              | GmbH             | Prefix "g"|
| `gUG`                | UG               | Prefix "g"|
| `gesellschaft mbH`   | GmbH             | Full spelling of GmbH |

These can be labeled as **🔍 Partial matches** using an explicit mapping dictionary.

---

#### ❌ Truly Unmatched Forms:

| Unmatched Form       | Reason |
|----------------------|--------|
| `e. V.`              | Verein (association), not a company form |
| `UG & Co. KG`        | Hybrid; may need a separate rule (UG + KG) |
| `Ltd.`               | Foreign legal form (UK) |
| `EI`                 | Not a standard form; possibly erroneous |
| `GmbH & Co. OHG`     | Complex hybrid; not cleanly matchable |
| `B.V.`               | Dutch legal form (*Besloten Vennootschap*) |
| `AG & Co. OHG`       | Hybrid; unclear structure |
| `Association`        | Generic term, not a legal form |
| `KG & Co. KG`        | Complex hybrid, not standard |

Keep these as **❌ Unmatched** for now — may require further discussion with the team before adding special handling for hybrids or international forms.

In [18]:
# Create a new dictionary for partial matched entries
partial_match_map = {
    'eGbR': 'GbR',
    'PartG mbB': 'PartGmbB',
    'gGmbH': 'GmbH',
    'gUG': 'UG',
    'gesellschaft mbH': 'GmbH'
}

In [19]:
# Update the enrich_match_quality function
def enrich_match_quality(df, lookup_dict, partial_match_map):
    def get_match_quality(legal_form):
        if pd.isna(legal_form) or str(legal_form).strip() == '':
            return '❓ Missing'

        legal_form_str = str(legal_form).strip()

        if legal_form_str in lookup_dict:
            return '✅ Exact'

        for key in lookup_dict:
            if key.lower() == legal_form_str.lower():
                return '✅ Exact'

        # Add partial match check here:
        if legal_form_str in partial_match_map:
            return '🔍 Partial'

        return '❌ Unmatched'

    df_enriched = df.copy()
    df_enriched['MATCH_QUALITY'] = df_enriched['LEGAL_FORM'].apply(get_match_quality)
    return df_enriched

#### Preview the result

In [21]:
# Apply to our dataset using the cleaned dictionary
df_enriched_partial = enrich_match_quality(df, lookup_dict, partial_match_map)

# View the result
preview_cols = ['NAME', 'LEGAL_FORM', 'MATCH_QUALITY']
available_cols = [col for col in preview_cols if col in df_enriched_partial.columns]

df_enriched_partial[available_cols].head(10)

,NAME,LEGAL_FORM,MATCH_QUALITY
0,Karl Roth mechanische Werkstätte GmbH,GmbH,✅ Exact
1,VACOM Vakuum Komponenten & Messtechnik GmbH,GmbH,✅ Exact
2,Landwirtschaftliche Erzeugergemeinschaft Berka...,eG,✅ Exact
3,Immobilienverwaltung Ottersbach GmbH & Co. KG,GmbH & Co. KG,✅ Exact
4,Bärhausen GmbH & Co. KG,GmbH & Co. KG,✅ Exact
5,R & S Szilagyi Grundstücksverwertungsgesellsch...,GmbH,✅ Exact
6,fischer/collegen Bietigheim-Bissingen GmbH & C...,GmbH & Co. KG,✅ Exact
7,Wirtschaftsprüfungsgesellschaft Frankfurt GmbH,GmbH,✅ Exact
8,DATA MODUL AG Produktion und Vertrieb von elek...,AG,✅ Exact
9,Hermann-Lietz-Schule Spiekeroog gemeinnützige ...,gGmbH,🔍 Partial


In [27]:
# Display partially matched records
partial_matches = df_enriched_partial[df_enriched_partial['MATCH_QUALITY'] == '🔍 Partial']
partial_matches[['NAME', 'LEGAL_FORM', 'MATCH_QUALITY']]

,NAME,LEGAL_FORM,MATCH_QUALITY
9,Hermann-Lietz-Schule Spiekeroog gemeinnützige ...,gGmbH,🔍 Partial
150,Diakonissenkrankenhaus Dessau gemeinnützige GmbH,gGmbH,🔍 Partial
245,Amani gUG,gUG,🔍 Partial
400,ALHA Seggewiß eGbR,eGbR,🔍 Partial
406,Erik van den Dolder und Christoph Ritzenthaler...,eGbR,🔍 Partial
...,...,...,...
999124,Mager Mobilien eGbR,eGbR,🔍 Partial
999126,Landwirtschaftsbetrieb Klemke eGbR,eGbR,🔍 Partial
999136,Arenbergstraße 13 eGbR,eGbR,🔍 Partial
999142,Reesenbüttler Redder 57 eGbR,eGbR,🔍 Partial


In [28]:
# Show summary statistics
print(f"Total records processed: {len(df_enriched):,}")

print("\nMatch Quality Distribution:")
match_stats = df_enriched_partial['MATCH_QUALITY'].value_counts()
for match_type, count in match_stats.items():
    percentage = (count / len(df_enriched)) * 100
    print(f"  {match_type}: {count:,} ({percentage:.1f}%)")

Total records processed: 999,967

Match Quality Distribution:
  ✅ Exact: 694,673 (69.5%)
  ❌ Unmatched: 196,911 (19.7%)
  ❓ Missing: 85,493 (8.5%)
  🔍 Partial: 22,890 (2.3%)


# 💼 Adding Business Types

In [36]:
def assign_business_type(df, lookup_dict, partial_match_map):
    def get_business_type(row):
        legal_form = row['LEGAL_FORM']
        match_quality = row['MATCH_QUALITY']
        
        if pd.isna(legal_form) or str(legal_form).strip() == '':
            return 'Private'  # Missing
        
        legal_form_str = str(legal_form).strip()
        
        if match_quality == '✅ Exact':
            # Match directly in lookup_dict
            for key in lookup_dict:
                if key.lower() == legal_form_str.lower():
                    return lookup_dict[key]
            return 'Private'  # Fallback
        
        elif match_quality == '🔍 Partial':
            corrected_form = partial_match_map.get(legal_form_str)
            if corrected_form:
                for key in lookup_dict:
                    if key.lower() == corrected_form.lower():
                        return lookup_dict[key]
            return 'Private'  # Fallback if correction fails

        # For ❌ Unmatched or ❓ Missing
        return 'Private'
    
    df['BUSINESS_TYPE'] = df.apply(get_business_type, axis=1)
    return df

In [37]:
# Apply and preview the result
df_fin = assign_business_type(df_enriched_partial, lookup_dict, partial_match_map)
df_fin.head(10)

,UNIQUE_KEY,NAME,YEAR,CITY,COUNTRY,STATUS,LEGAL_FORM,ISIC,REGISTRATION,LIQUIDATION,TERMINATION,REVENUES,TOTAL ASSETS,MATCH_QUALITY,BUSINESS_TYPE
0,"1,29076E+11",Karl Roth mechanische Werkstätte GmbH,2024.0,fürth,DE,active,GmbH,2592.0,1989.0,NaN,NaN,990.000.000.000.000,865.389.500.000.000,✅ Exact,Private
1,"1,20775E+13",VACOM Vakuum Komponenten & Messtechnik GmbH,2024.0,großlöbichau,DE,active,GmbH,2651.0,1993.0,NaN,NaN,58.401.908.050.000.000,56.201.332.840.000.000,✅ Exact,Private
2,1339576043,Landwirtschaftliche Erzeugergemeinschaft Berka...,2024.0,bismark (altmark),DE,active,eG,150.0,1991.0,NaN,NaN,2.400.000.000.000.000,81.183.970.000.000,✅ Exact,Hybrid
3,"1,15311E+11",Immobilienverwaltung Ottersbach GmbH & Co. KG,2024.0,bonn,DE,active,GmbH & Co. KG,6820.0,2015.0,NaN,NaN,710.000.000.000.000,3.213.887.520.000.000,✅ Exact,Hybrid
4,"1,13539E+11",Bärhausen GmbH & Co. KG,2024.0,lauterbach,DE,active,GmbH & Co. KG,2822.0,1965.0,NaN,NaN,4.500.000.000.000.000,2.404.823.060.000.000,✅ Exact,Hybrid
5,"1,2791E+13",R & S Szilagyi Grundstücksverwertungsgesellsch...,2004.0,bad säckingen,DE,active,GmbH,6810.0,2000.0,NaN,NaN,1.600.000.000.000.000,1.485.923.580.000.000,✅ Exact,Private
6,"1,17019E+13",fischer/collegen Bietigheim-Bissingen GmbH & C...,2024.0,bietigheim-bissingen,DE,active,GmbH & Co. KG,6920.0,2011.0,NaN,NaN,1.700.000.000.000.000,654.408.070.000.000,✅ Exact,Hybrid
7,"1,26031E+11",Wirtschaftsprüfungsgesellschaft Frankfurt GmbH,2004.0,frankfurt a. main,DE,terminated,GmbH,NaN,1960.0,2010.0,2011.0,NaN,355.097.570.000.000,✅ Exact,Private
8,"1,28033E+12",DATA MODUL AG Produktion und Vertrieb von elek...,2024.0,münchen,DE,active,AG,2610.0,1999.0,NaN,NaN,226.208.000.000.000.000,210.071.000.000.000.000,✅ Exact,Public
9,"1,2266E+11",Hermann-Lietz-Schule Spiekeroog gemeinnützige ...,2024.0,spiekeroog,DE,active,gGmbH,8510.0,1984.0,NaN,NaN,9.400.000.000.000.000,8.682.542.510.000.000,🔍 Partial,Private


In [38]:
print("\nBusiness Type Distribution:")
business_stats = df_fin['BUSINESS_TYPE'].value_counts()
for biz_type, count in business_stats.items():
    percentage = (count / len(df_enriched)) * 100
    print(f"  {biz_type}: {count:,} ({percentage:.1f}%)")


Business Type Distribution:
  Private: 919,701 (92.0%)
  Hybrid: 74,521 (7.5%)
  Public: 5,745 (0.6%)
